# Create a search index in Azure AI Search using the Azure SDK for Python

This Jupyter Notebook steps through creating, loading, and querying an index in Azure AI Search index by calling the azure-search-documents library in the Azure SDK for Python. 

In [55]:
# Uncomment the next line if you need to install the azure-search-documents client library:
%pip install azure-search-documents --pre --upgrade
%pip show azure-search-documents
%pip install python-dotenv

import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient

from azure.search.documents.indexes.models import (  
    CorsOptions,
    SearchIndex,  
    SearchField,  
    SemanticSearch,
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,
    ComplexField,
    SearchIndex,  
    SemanticConfiguration,  
    SemanticPrioritizedFields,  
    SemanticField 
)

Note: you may need to restart the kernel to use updated packages.
Name: azure-search-documentsNote: you may need to restart the kernel to use updated packages.

Version: 11.4.0
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: c:\Users\manuchadha\OneDrive - Microsoft\Manu_Microsoft\demo-data\aoai-openai\aoai-code\.venv\Lib\site-packages
Requires: azure-common, azure-core, isodate
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In this step, initialize the search client used to make each request. Provide the name and admin API key of your search service. If you get ConnectionError "Failed to establish a new connection", verify that the api-key is a primary or secondary admin key, and not a query key.

In [104]:
# Set the service endpoint and API key from the environment

service_name = "aisearchmcuksouth"
admin_key = "sguckbZFk6EbbP8Azrosy1temBCqumUdCSW73ZLJ5JAzSeBNqRps"

index_name = "hotel-finder-semantic"

# Create an SDK client
endpoint = "https://aisearchmcuksouth.search.windows.net".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))


In the third cell, the index "invoices-finder" will be deleted if it previously existed.

In [105]:
# Delete the index if it exists
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)


Index hotel-finder-semantic Deleted


Execute this for semantic ranking part of test

In [106]:


# Specify the index schema
name = index_name
fields = [
        SimpleField(name="HotelId", type=SearchFieldDataType.String, key=True),
        SearchableField(name="HotelName", type=SearchFieldDataType.String, sortable=True),
        SearchableField(name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"),
        SearchableField(name="Description_fr", type=SearchFieldDataType.String, analyzer_name="fr.lucene"),
        SearchableField(name="Category", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

        SearchableField(name="Tags", collection=True, type=SearchFieldDataType.String, facetable=True, filterable=True),

        SimpleField(name="ParkingIncluded", type=SearchFieldDataType.Boolean, facetable=True, filterable=True, sortable=True),
        SimpleField(name="LastRenovationDate", type=SearchFieldDataType.DateTimeOffset, facetable=True, filterable=True, sortable=True),
        SearchableField(name="Rating", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),

        ComplexField(name="Address", fields=[
            SearchableField(name="StreetAddress", type=SearchFieldDataType.String),
            SearchableField(name="City", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="StateProvince", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="PostalCode", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
            SearchableField(name="Country", type=SearchFieldDataType.String, facetable=True, filterable=True, sortable=True),
        ])
    ]

semantic_config = SemanticConfiguration(
    name="my-semantic-config",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="HotelName"),
        keywords_fields=[SemanticField(field_name="Category")],
        content_fields=[SemanticField(field_name="Description")]
    )
)

semantic_search = SemanticSearch(configurations=[semantic_config])
suggester = [{'name': 'sg', 'source_fields': ['Tags', 'Address/City', 'Address/Country']}]
scoring_profiles = []

execute for semantic search

In [107]:
index = SearchIndex(
    name=name,
    fields=fields,
    semantic_search = semantic_search,
    scoring_profiles=scoring_profiles,
    suggesters=suggester)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

Index hotel-finder-semantic created


Next, provide documents that conform to the index schema. Specify an upload action for each document.

In [86]:
documents = [
    {
    "@search.action": "upload",
    "HotelId": "1",
    "HotelName": "Secret Point Motel",
    "Description": "The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.",
    "Description_fr": "L'hôtel est idéalement situé sur la principale artère commerciale de la ville en plein cœur de New York. A quelques minutes se trouve la place du temps et le centre historique de la ville, ainsi que d'autres lieux d'intérêt qui font de New York l'une des villes les plus attractives et cosmopolites de l'Amérique.",
    "Category": "Boutique",
    "Tags": [ "pool", "air conditioning", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1970-01-18T00:00:00Z",
    "Rating": "3.60",
    "Address": {
        "StreetAddress": "677 5th Ave",
        "City": "New York",
        "StateProvince": "NY",
        "PostalCode": "10022",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "2",
    "HotelName": "Twin Dome Motel",
    "Description": "The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Category": "Boutique",
    "Tags": [ "pool", "free wifi", "concierge" ],
    "ParkingIncluded": "false",
    "LastRenovationDate": "1979-02-18T00:00:00Z",
    "Rating": "3.60",
    "Address": {
        "StreetAddress": "140 University Town Center Dr",
        "City": "Sarasota",
        "StateProvince": "FL",
        "PostalCode": "34243",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "3",
    "HotelName": "Triple Landscape Hotel",
    "Description": "The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.",
    "Description_fr": "L'hôtel est situé dans une place du XIXe siècle, qui a été agrandie et rénovée aux plus hautes normes architecturales pour créer un hôtel moderne, fonctionnel et de première classe dans lequel l'art et les éléments historiques uniques coexistent avec le confort le plus moderne.",
    "Category": "Resort and Spa",
    "Tags": [ "air conditioning", "bar", "continental breakfast" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "2015-09-20T00:00:00Z",
    "Rating": "4.80",
    "Address": {
        "StreetAddress": "3393 Peachtree Rd",
        "City": "Atlanta",
        "StateProvince": "GA",
        "PostalCode": "30326",
        "Country": "USA"
        }
    },
    {
    "@search.action": "upload",
    "HotelId": "4",
    "HotelName": "Sublime Cliff Hotel",
    "Description": "Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.",
    "Description_fr": "Le sublime Cliff Hotel est situé au coeur du centre historique de sublime dans un quartier extrêmement animé et vivant, à courte distance de marche des sites et monuments de la ville et est entouré par l'extraordinaire beauté des églises, des bâtiments, des commerces et Monuments. Sublime Cliff fait partie d'un Palace 1800 restauré avec amour.",
    "Category": "Boutique",
    "Tags": [ "concierge", "view", "24-hour front desk service" ],
    "ParkingIncluded": "true",
    "LastRenovationDate": "1960-02-06T00:00:00Z",
    "Rating": "4.60",
    "Address": {
        "StreetAddress": "7400 San Pedro Ave",
        "City": "San Antonio",
        "StateProvince": "TX",
        "PostalCode": "78216",
        "Country": "USA"
        }
    }
]

Formulate the request. This upload_documents request targets the docs collection of the hotels-quickstart index and pushes the documents provided in the previous step into the search index.

In [87]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

Upload of new document succeeded: True


In [88]:
results =  search_client.search(query_type='simple',
    search_text="*" ,
    select='HotelName,Description', 
    include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print(result["HotelName"])

Total Documents Matching Query: 4
1.0
Triple Landscape Hotel
1.0
Twin Dome Motel
1.0
Sublime Cliff Hotel
1.0
Secret Point Motel


In [90]:
results =  search_client.search(query_type='simple',search_text="hotel" ,select='HotelId,HotelName,Rating', order_by='Rating desc', include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {} - {} rating".format(result["HotelId"], result["HotelName"], result["Rating"]))

Total Documents Matching Query: 4
1.0144587
3: Triple Landscape Hotel - 4.80 rating
0.9284746
4: Sublime Cliff Hotel - 4.60 rating
0.51237863
1: Secret Point Motel - 3.60 rating
0.509938
2: Twin Dome Motel - 3.60 rating


In [92]:
results = search_client.search(search_text="motel", select='HotelId,HotelName,Rating',  order_by='Rating desc', include_total_count=True)
print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {} - {} rating".format(result["HotelId"], result["HotelName"], result["Rating"]))

Total Documents Matching Query: 2
0.6099695
2: Twin Dome Motel - 3.60 rating
0.25316024
1: Secret Point Motel - 3.60 rating


In [93]:
results = search_client.search(search_text="sublime", search_fields=['HotelName'], select='HotelId,HotelName', include_total_count=True)
print ('Total Documents Matching Query:', results.get_count())

for result in results:
    print(result["@search.score"])
    print("{}: {}".format(result["HotelId"], result["HotelName"]))

Total Documents Matching Query: 1
0.6099695
4: Sublime Cliff Hotel


In [94]:
results = search_client.search(search_text="wifi", include_total_count=True, select='HotelId,HotelName,Tags')
print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print(result["@search.score"])
    print("{}: {}: {}".format(result["HotelId"], result["HotelName"], result["Tags"]))

Total Documents Matching Query: 1
0.78019357
2: Twin Dome Motel: ['pool', 'free wifi', 'concierge']


In [96]:
results =  search_client.search(query_type='simple',
    search_text="what hotel has a good restaurant on site" ,
    select='HotelName,HotelId,Description')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    

2.1393623
None
Sublime Cliff Hotel
1.9309065
None
Twin Dome Motel
1.5589908
None
Triple Landscape Hotel
0.7704947
None
Secret Point Motel


In [98]:
results =  search_client.search(query_type='semantic',semantic_configuration_name='my-semantic-config',
    search_text="what hotel has a good restaurant on site" ,
    select='HotelName,HotelId,Description')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])
    

1.5589908
1.8807636499404907
Triple Landscape Hotel
2.1393623
1.527956485748291
Sublime Cliff Hotel
0.7704947
1.3261334896087646
Secret Point Motel
1.9309065
1.0473345518112183
Twin Dome Motel


In [100]:
results =  search_client.search(query_type='simple',
    search_text="what hotel stands out for its gastronomic excellence", 
    select='HotelName,Description,Category')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])

2.3295155
None
Triple Landscape Hotel
0.9284746
None
Sublime Cliff Hotel
0.51237863
None
Secret Point Motel
0.509938
None
Twin Dome Motel


In [102]:
results =  search_client.search(query_type='semantic', semantic_configuration_name='my-semantic-config',
    search_text="which restaurant has rating of 4 or over?", 
    select='HotelName,Description,Category', query_caption='extractive')

for result in results:
    print(result["@search.score"])
    print(result["@search.reranker_score"])
    print(result["HotelName"])

0.26301134
0.9669117331504822
Triple Landscape Hotel
1.6842916
0.061580806970596313
Twin Dome Motel


In [103]:
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

Index hotel-finder-semantic Deleted


In [54]:
try:
    result = admin_client.get_index(index_name)
    print (result)
except Exception as ex:
    print (ex)

() No index with the name 'hotel-finder-semantic' was found in the service 'aisearchmcuksouth'.
Code: 
Message: No index with the name 'hotel-finder-semantic' was found in the service 'aisearchmcuksouth'.
